In [ ]:
%load_ext autoreload
%autoreload 2

import anndata as ad
import matplotlib.pyplot as plt
import pandas as pd

from commando import ComManDo
from commando.utilities import visualize_mapping

# Read Data

In [ ]:
data_mod1 = ad.read_h5ad('OPSCA/openproblems_bmmc_multiome_phase1/openproblems_bmmc_multiome_phase1.censor_dataset.output_mod1.h5ad')
data_mod2 = ad.read_h5ad('OPSCA/openproblems_bmmc_multiome_phase1/openproblems_bmmc_multiome_phase1.censor_dataset.output_mod2.h5ad')

# data_mod1 = ad.read_h5ad('OPSCA/openproblems_bmmc_cite_phase1/openproblems_bmmc_cite_phase1.censor_dataset.output_mod1.h5ad')
# data_mod2 = ad.read_h5ad('OPSCA/openproblems_bmmc_cite_phase1/openproblems_bmmc_cite_phase1.censor_dataset.output_mod2.h5ad')

# Run Mapping

In [ ]:
commando_out = (
    ComManDo(
        distance_mode='spearman',
        two_step_aggregation='cell_cycle',
        
        two_step_num_partitions=None,
        epoch_pd=1000,
        two_step_pd_large=1000,
        
        log_pd=1,
        two_step_log_pd=1,
    )
    .fit_transform([data_mod1, data_mod2])
)

# Evaluate

In [ ]:
plt.subplots(figsize=(12, 6))
plt.subplot(1, 2, 1)
visualize_mapping(commando_out, 0)
plt.subplot(1, 2, 2)
visualize_mapping(commando_out, 1)